<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/jc/c1_crawling_modify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## 무조건 해야하는 부분 1
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

# headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}

# 전체 과정 사이클

In [17]:
## colab

url = "https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

In [2]:
## local

url = "https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01"
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument('headless') # headless 모드 설정
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")

# 속도 향상을 위한 옵션 해제
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2, 'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}}   
options.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome(r'C:\Users\yunjc\chromedriver_win32\chromedriver.exe', options=options)

C:\Users\yunjc\AppData\Local\Temp\ipykernel_37632\3536521958.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\yunjc\chromedriver_win32\chromedriver.exe', options=options)


In [9]:
pre_df = pd.read_csv(r"G:\내 드라이브\KB인턴관련\crawl_1_euckr.csv", encoding='euc-kr')
pre_df.tail()

,category,lawName,ENG,KOR
68167,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,Article 7 (Demand for Materials Related to Con...,제7조(계약에 관한 자료 등의 요구)\n① 기획재정부장관은 정부조달협정가입국으로부터...
68168,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,Article 8 (Reporting Results of Contracts)\n(1...,제8조(계약실적보고)\n① 특례규정 제25조제1항에 따른 계약실적 보고서의 내용은 ...
68169,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,ADDENDUM <Ordinance of the Ministry of Strateg...,"부칙 <제358호, 2013.8.7.>\n이 규칙은 공포한 날부터 시행한다."
68170,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,ADDENDA <Ordinance of the Ministry of Strategy...,"부칙 <제360호, 2013.9.17.> (전자조달의 이용 및 촉진에 관한 법률 시..."
68171,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,ADDENDUM <Ordinance of the Ministry of Strateg...,"부칙 <제492호, 2015.7.20.>\n이 규칙은 정부조달에 관한 협정 개정 의..."


In [10]:
def wait_driver(xpath, t=10):
    WebDriverWait(driver, t).until(EC.presence_of_element_located((By.XPATH, xpath)))
    
def tab_move():
    driver.switch_to.window(driver.window_handles[-1])
    ko_url = driver.current_url.replace("ENG", "KOR")
    driver.get(ko_url)
    wait_driver('//*[@id="lawViewContent"]')
    driver.switch_to.frame("lawViewContent")

def tab_close():
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    driver.switch_to.frame("indexFrame")

def parsing_laws():
    lang_pairs = {"eng": [], "kor": []}
    for i, p in enumerate(driver.find_elements(By.CSS_SELECTOR, '#body > div > table > tbody > tr > td')):
        if i<4: continue
        for line in p.text.split("\n"):
            if i%2==0:
                lang_pairs["eng"].append(line)
            else:
                lang_pairs["kor"].append(line)

In [ ]:
## crawler total

driver.get(url)

categories = [f'tree_{i}_a' for i in range(2, 47)]
# category loop
base_df = pd.DataFrame()
for j, c in enumerate(categories):
    category = driver.find_element(By.ID, c)
    cate_list = category.text.split(" ")
    cate_name, num_of_laws = cate_list[1], int(cate_list[2].strip('()'))
    driver.execute_script("arguments[0].click();", category)

    # pages
    driver.switch_to.frame('indexFrame')
    # pages = driver.find_elements(By.CSS_SELECTOR, "#container > div > div.paging > a")
    pages = [f'//*[@id="container"]/div/div[3]/a[{k}]' for k in range(3, 13)]
    page_idx = 0
    complete_num = 0
    print(f"{j+1}. {cate_name} 시작...")
    while True:
    # laws loop
        wait_driver('//*[@id="1"]/td[2]')
        laws = driver.find_elements(By.CSS_SELECTOR, '#list1 > tbody > tr > td > p > a')
        page_parsing_num = 0
        for law in laws:
            law_name = law.text.split('\n')[0]
            law.send_keys(Keys.ENTER)

            tab_move()
            wait_driver('/html')

            # Parsing
            eng, kor = [], []
            contents = driver.find_elements(By.CSS_SELECTOR, '#body > div > table > tbody > tr > td')
            for i, c in enumerate(contents[4:-2]):
                if i%2==0:
                    eng.append(c.text)
                else:
                    kor.append(c.text)
            if len(eng)!=len(kor):
                raise Exception(f"Different Length ENG{len(eng)}, KOR{len(kor)}")
            else:
                df = pd.DataFrame({"category":[cate_name]*len(eng), "lawName":[law_name]*len(eng), "ENG":eng, "KOR":kor})
                page_parsing_num+=df.shape[0]
                # print(f"{law_name} 파싱 완료 : {len(eng)} 개")

            # fininsh parsing
            tab_close()
            base_df = pd.concat([base_df, df], axis=0)
        print(f"\t{page_idx+1}페이지 파싱 완료: {page_parsing_num}")
        complete_num += len(laws)
        if complete_num < num_of_laws:
            driver.find_element(By.XPATH, pages[page_idx]).click()
            page_idx+=1
        else:
            break
    driver.get(url)
    wait_driver('//*[@id="tree_1"]')
driver.quit()

In [7]:
cate_pass, page_pass, law_pass = True, True, True

In [8]:
## crawler (중단 지점 부터)

driver.get(url)

categories = [f'tree_{i}_a' for i in range(2, 47)]
# category loop
base_df = pd.DataFrame()
for j, c in enumerate(categories):
    category = driver.find_element(By.ID, c)
    cate_list = category.text.split(" ")
    cate_name, num_of_laws = cate_list[1], int(cate_list[2].strip('()'))
    if cate_name=='재정ㆍ경제일반':
        cate_pass=False
        print('category 중단점 도착')
    if cate_pass: continue

    driver.execute_script("arguments[0].click();", category)

    # pages
    driver.switch_to.frame('indexFrame')
    # pages = driver.find_elements(By.CSS_SELECTOR, "#container > div > div.paging > a")
    pages = [f'//*[@id="container"]/div/div[3]/a[{k}]' for k in range(3, 13)]
    page_idx = 0
    complete_num = 0
    print(f"{j+1}. {cate_name} 시작...")
    while True:
    # laws loop
        wait_driver('//*[@id="1"]/td[2]')
        laws = driver.find_elements(By.CSS_SELECTOR, '#list1 > tbody > tr > td > p > a')
        page_parsing_num = 0
        for law in laws:
            law_name = law.text.split('\n')[0]
            if law_name=='특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행령 특례규정':
                law_pass = False
                print('\t법령 중단점 도착\n새로운 파싱 시작')
            if law_pass: continue
            law.send_keys(Keys.ENTER)

            tab_move()
            wait_driver('/html')

            # Parsing
            eng, kor = [], []
            contents = driver.find_elements(By.CSS_SELECTOR, '#body > div > table > tbody > tr > td')
            for i, c in enumerate(contents[4:-2]):
                if i%2==0:
                    eng.append(c.text)
                else:
                    kor.append(c.text)
            if len(eng)!=len(kor):
                raise Exception(f"Different Length ENG{len(eng)}, KOR{len(kor)}")
            else:
                df = pd.DataFrame({"category":[cate_name]*len(eng), "lawName":[law_name]*len(eng), "ENG":eng, "KOR":kor})
                page_parsing_num+=df.shape[0]
                # print(f"{law_name} 파싱 완료 : {len(eng)} 개")

            # fininsh parsing
            tab_close()
            base_df = pd.concat([base_df, df], axis=0)
        print(f"\t{page_idx+1}페이지 파싱 완료: {page_parsing_num}")
        complete_num += len(laws)
        if complete_num < num_of_laws:
            driver.find_element(By.XPATH, pages[page_idx]).click()
            page_idx+=1
        else:
            break
    driver.get(url)
    wait_driver('//*[@id="tree_1"]')
driver.quit()


category 중단점 도착
19. 재정ㆍ경제일반 시작...
	1페이지 파싱 완료: 0
	2페이지 파싱 완료: 0
	3페이지 파싱 완료: 0
	4페이지 파싱 완료: 0
	법령 중단점 도착
새로운 파싱 시작
	5페이지 파싱 완료: 140
	6페이지 파싱 완료: 903
20. 내국세 시작...
	1페이지 파싱 완료: 2434
	2페이지 파싱 완료: 2947
	3페이지 파싱 완료: 1033
21. 관세 시작...
	1페이지 파싱 완료: 1640
22. 담배·인삼 시작...
	1페이지 파싱 완료: 230
23. 통화ㆍ국채ㆍ금융 시작...
	1페이지 파싱 완료: 1037
	2페이지 파싱 완료: 1231
	3페이지 파싱 완료: 1701
	4페이지 파싱 완료: 1665
	5페이지 파싱 완료: 2361
	6페이지 파싱 완료: 774
24. 농업 시작...
	1페이지 파싱 완료: 1389
	2페이지 파싱 완료: 1431
	3페이지 파싱 완료: 1151
	4페이지 파싱 완료: 517
25. 축산 시작...
	1페이지 파싱 완료: 1060
	2페이지 파싱 완료: 347
26. 산림 시작...
	1페이지 파싱 완료: 1607
	2페이지 파싱 완료: 227
27. 수산 시작...
	1페이지 파싱 완료: 1203
	2페이지 파싱 완료: 487
28. 상업ㆍ무역ㆍ공업 시작...
	1페이지 파싱 완료: 1231
	2페이지 파싱 완료: 1529
	3페이지 파싱 완료: 2075
	4페이지 파싱 완료: 1286
	5페이지 파싱 완료: 1304
	6페이지 파싱 완료: 1640
	7페이지 파싱 완료: 801
29. 공업규격·계량 시작...
	1페이지 파싱 완료: 933
30. 공업소유권 시작...
	1페이지 파싱 완료: 1521
	2페이지 파싱 완료: 519
31. 에너지이용ㆍ광업 시작...
	1페이지 파싱 완료: 1449
	2페이지 파싱 완료: 491
32. 전기ㆍ가스 시작...
	1페이지 파싱 완료: 1622
	2페이지 파싱 완료: 39
33. 국토개발·도시 시작...
	1페이지 파싱 완료: 

In [13]:
total_df = pd.concat([pre_df, base_df], axis=0)
total_df.reset_index(drop=True, inplace=True)

In [ ]:
# total_df.to_csv(r"G:\내 드라이브\KB인턴관련\crawl_total_local.csv", index=False, encoding='euc-kr')

# Parsing 로직 다시 짜기

In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

## local

url = "https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01"
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
# options.add_argument('headless') # headless 모드 설정
# options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
# options.add_argument("disable-gpu")
# options.add_argument("disable-infobars")
# options.add_argument("--disable-extensions")

# 속도 향상을 위한 옵션 해제
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2, 'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}}   
options.add_experimental_option('prefs', prefs)

# driver = webdriver.Chrome(r'C:\Users\yunjc\chromedriver_win32\chromedriver.exe', options=options)
driver = webdriver.Chrome(r'C:\Users\yunjc\chromedriver_win32\chromedriver.exe')

driver.get(url)

C:\Users\yunjc\AppData\Local\Temp\ipykernel_9664\1550359854.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\yunjc\chromedriver_win32\chromedriver.exe')


In [2]:
driver.window_handles

['6906F25413CD87C0D629FE832077176F', '06AD2282C7DC21B8FF3A219E169C098F']

In [3]:
driver.switch_to.window(driver.window_handles[-1])
driver.switch_to.frame("lawViewContent")

In [4]:
hang = driver.find_elements(By.CLASS_NAME, "hang")
ho = driver.find_elements(By.CLASS_NAME, "ho")
none = driver.find_elements(By.CLASS_NAME, "none")

for i, target in enumerate([none, ho, hang]):
    if i==0: print('none')
    elif i==1: print('ho')
    else: print('hang')
    for v in target:
        print(v.text)
    print('===================================')

none
The purpose of this Decree is to provide for matters delegated by the Secured Bond Trust Act.
이 영은 담보부사채신탁법에서 위임된 사항을 규정함을 목적으로 한다.
이 영은 공포한 날부터 시행한다.부칙(전자적민원처리를위한가석방자관리규정등중개정령)<제18312호,2004.3.17>
이 영은 공포한 날부터 시행한다.
부칙<제20653호,2008.2.29>
이 영은 공포한 날부터 시행한다. 다만, 부칙 제2조에 따라 개정되는 대통령령 중 이 영의 시행 전에 공포되었으나 시행일이 도래하지 아니한 대통령령을 개정한 부분은 각각 해당 대통령령의 시행일부터 시행한다.
ho
hang
(1) Whenever imposing a fine for negligence under the provisions of Article 105 (3) of the Secured Bond Trust Act, an investigation shall be conducted to verify the relevant offense, and then a written notice shall be served on the person subject to the disposition of a fine for negligence to pay it specifying the facts of the offense, the amount of the fine for negligence, etc.
(2) The Financial Services Commission shall give the person subject to the disposition of a fine for negligence an opportunity to make his/her statement orally or in writing within a prescribed period of time of ten days or more, whenever it intends t

In [17]:
driver.window_handles

['CEB822E90186A131B8079EA82213C3F3', 'F55809EB4F0EBF6F28BA4EE5ED182058']

In [47]:
driver.switch_to.window(driver.window_handles[-1])
driver.switch_to.frame('lawViewContent')

In [22]:
import re

def parsing(cate, law):
    eng_q, kor_q = [], []
    pattern = '[^ㄱ-ㅣ가-힣]'
    contents = []
    for key in ["none", "ho", "hang"]:
        contents+=driver.find_elements(By.CLASS_NAME, key)
    cur_eng, cur_kor = [], []
    print(len(contents))
    for c in contents:
        cur_txt = c.text
        if re.sub(pattern, '', cur_txt)=='':    # 영어 탐색
            if len(cur_eng) >= len(cur_kor):
                cur_eng.append(cur_txt)
                # eng_num+=1
            else:
                continue
        else:
            if len(cur_kor) < len(cur_eng):
                cur_kor.append(cur_txt)
                # kor_num+=1
            else:
                continue

        # if kor_num==0:
        # eng_q+=cur_eng
        # kor_q+=cur_kor
            # cur_eng, cur_kor = [], []
            
    return pd.DataFrame(zip([cate]*len(cur_eng), [law]*len(cur_kor), cur_eng, cur_kor), columns=["category", "lawName", "ENG", "KOR"])
    
result = parsing("분류", "법령명")
result

5099


,category,lawName,ENG,KOR
0,헌법,개성공업지구 지원에 관한 법률,The purpose of this Act is to efficiently reha...,이 법은 재정적 어려움으로 인하여 파탄에 직면해 있는 채무자에 대하여 채권자ㆍ주주ㆍ...
1,헌법,개성공업지구 지원에 관한 법률,"In the application of this Act, foreigners and...",외국인 또는 외국법인은 이 법의 적용에 있어서 대한민국 국민 또는 대한민국 법인과 ...
2,헌법,개성공업지구 지원에 관한 법률,When it is deemed necessary for avoiding any s...,법원은 현저한 손해 또는 지연을 피하기 위하여 필요하다고 인정하는 때에는 직권으로 ...
3,헌법,개성공업지구 지원에 관한 법률,"In the procedures provided for in this Act, co...",이 법에 의한 절차에서 법원은 서로 법률상의 협조를 구할 수 있다.
4,헌법,개성공업지구 지원에 관한 법률,Any objection to a judgment made pursuant to t...,이 법의 규정에 의한 재판에 대한 불복은 서면으로 하여야 한다.
...,...,...,...,...
2539,헌법,개성공업지구 지원에 관한 법률,(2) Where any of the following persons commits...,② 생략
2540,헌법,개성공업지구 지원에 관한 법률,(3) A person who commits an act of collection ...,① (시행일) 이 법은 공포한 날부터 시행한다.
2541,헌법,개성공업지구 지원에 관한 법률,(1) (Enforcement Date) This Act shall enter in...,② (경과조치) 이 법 시행 전에 회생절차개시 후에 채무자의 업무 및 재산에 관하여...
2542,헌법,개성공업지구 지원에 관한 법률,(2) Omitted.,① 제611조제5항의 개정규정은 같은 개정규정 시행 후 최초로 신청하는 개인회생사건...


In [25]:
for i in result.index:
    print(result.loc[i, "ENG"])
    print(result.loc[i, "KOR"])
    print("________________________________")
    
## Where a rehabilitation proposal is not submitted within the period or extended period respectively, set by the court or all of the draft;

The purpose of this Act is to efficiently rehabilitate debtors facing distress due to financial difficulties, and their businesses through coordination of legal relations among interested persons, including creditors, shareholders and equity holders, etc. and to fairly realize and divide properties of debtors who are deemed difficult to rehabilitate.
이 법은 재정적 어려움으로 인하여 파탄에 직면해 있는 채무자에 대하여 채권자ㆍ주주ㆍ지분권자 등 이해관계인의 법률관계를 조정하여 채무자 또는 그 사업의 효율적인 회생을 도모하거나, 회생이 어려운 채무자의 재산을 공정하게 환가ㆍ배당하는 것을 목적으로 한다.
________________________________
In the application of this Act, foreigners and foreign corporations shall have the same status as that of peoples of the Republic of Korea or corporations of the Republic of Korea.
외국인 또는 외국법인은 이 법의 적용에 있어서 대한민국 국민 또는 대한민국 법인과 동일한 지위를 가진다.
________________________________
When it is deemed necessary for avoiding any substantial damage or delay, the court may, ex officio, transfer a rehabilitation case, bankruptcy case, or individual rehabilitation case to any of the f

In [28]:
contents = driver.find_elements(By.CSS_SELECTOR, '#body > div > table > tbody > tr > td')
for c in contents:
    print(c.text)
    print("______________________________")

DEBTOR REHABILITATION AND BANKRUPTCY ACT
______________________________
채무자 회생 및 파산에 관한 법률
______________________________
 Act No. 7428, Mar. 31, 2005
Amended by Act No. 7895, Mar. 24, 2006
Act No. 7894, Mar. 24, 2006
Act No. 7892, Mar. 24, 2006
Act No. 8138, Dec. 30, 2006
Act No. 8635, Aug. 3, 2007
Act No. 8814, Dec. 27, 2007
Act No. 8829, Dec. 31, 2007
Act No. 8863, Feb. 29, 2008
Act No. 9346, Jan. 30, 2009
Act No. 9804, Oct. 21, 2009
Act No. 10219, Mar. 31, 2010
Act No. 10281, May 14, 2010
Act No. 10303, May 17, 2010
Act No. 10366, jun. 10, 2010
Act No. 10682, May 19, 2011
Act No. 11828, May 28, 2013
Act No. 12153, Jan. 1, 2014
Act No. 12595, May 20, 2014
Act No. 12591, May 20, 2014
Act No. 12783, Oct. 15, 2014
Act No. 12892, Dec. 30, 2014
Act No. 14177, May 29, 2016
Act No. 14476, Dec. 27, 2016
Act No. 14472, Dec. 27, 2016
Act No. 15158, Dec. 12, 2017
Act No. 16568, Aug. 27, 2019
Act No. 16652, Nov. 26, 2019
Act No. 16920, Feb. 4, 2020
Act No. 17088, Mar. 24, 2020
Act No. 17364, ju

In [34]:
contents[6].find_elements(By.CLASS_NAME, "none")[0].text

'The purpose of this Act is to efficiently rehabilitate debtors facing distress due to financial difficulties, and their businesses through coordination of legal relations among interested persons, including creditors, shareholders and equity holders, etc. and to fairly realize and divide properties of debtors who are deemed difficult to rehabilitate.'

In [41]:
contents = driver.find_elements(By.CSS_SELECTOR, '#body > div > table > tbody > tr > td')

eng_line, kor_line = [], []
for i in range(0, len(contents), 2):
    cur_eng, cur_kor = [], []
    eng_idx, kor_idx = i, i+1
    if contents[eng_idx].text=='' or contents[kor_idx].text=='':
        print(contents[eng_idx].text)
        continue
    for key in ["none", "ho", "hang"]:
        cur_eng = contents[eng_idx].find_elements(By.CLASS_NAME, key)
        cur_kor = contents[kor_idx].find_elements(By.CLASS_NAME, key)
        if len(cur_eng)==len(cur_kor):
            eng_line+=cur_eng
            kor_line+=cur_kor
print(len(eng_line), len(kor_line))


Article 286 (Discontinuation before Authorization Granted for Rehabilitation Plan)
(1) In each of the following cases, the court shall ex officio determine to discontinue rehabilitation procedures:
1. Where a rehabilitation proposal is not submitted within the period or extended period respectively, set by the court or all of the draft;
2. Where a rehabilitation proposal is voted against or is not resolved upon at the assembly of related persons within two months or the extended period from the first day of the assembly of related persons;
3. A rehabilitation proposal is not resolved upon at the assembly of related persons within the period provided for in the provisions of Article 239 (3);
4. When it is decided to place a rehabilitation proposal in a written resolution pursuant to the provisions of Article 240 (1), however the rehabilitation proposal is not resolved upon according to the written resolution: Provided, That when the hearing continuation date provided for in the provisi

In [42]:
for e, k in zip(eng_line, kor_line):
    print(e.text)
    print(k.text)
    print("_____________________________________")

The purpose of this Act is to efficiently rehabilitate debtors facing distress due to financial difficulties, and their businesses through coordination of legal relations among interested persons, including creditors, shareholders and equity holders, etc. and to fairly realize and divide properties of debtors who are deemed difficult to rehabilitate.
이 법은 재정적 어려움으로 인하여 파탄에 직면해 있는 채무자에 대하여 채권자ㆍ주주ㆍ지분권자 등 이해관계인의 법률관계를 조정하여 채무자 또는 그 사업의 효율적인 회생을 도모하거나, 회생이 어려운 채무자의 재산을 공정하게 환가ㆍ배당하는 것을 목적으로 한다.
_____________________________________
In the application of this Act, foreigners and foreign corporations shall have the same status as that of peoples of the Republic of Korea or corporations of the Republic of Korea.
외국인 또는 외국법인은 이 법의 적용에 있어서 대한민국 국민 또는 대한민국 법인과 동일한 지위를 가진다.
_____________________________________
1. The general jurisdiction of the debtor;
1. 채무자의 보통재판적이 있는 곳
_____________________________________
2. The place where the debtor's principal office or place of business is located, or the

In [6]:
driver.switch_to.window(driver.window_handles[-1])
driver.switch_to.frame('lawViewContent')

In [7]:
import re

def parsing(cate, law):
    contents = driver.find_elements(By.CSS_SELECTOR, '#body > div > table > tbody > tr > td')

    eng_line, kor_line = [], []
    for i in range(0, len(contents), 2):
        cur_eng, cur_kor = [], []
        eng_idx, kor_idx = i, i+1
        if contents[eng_idx].text=='' or contents[kor_idx].text=='':
            # print(contents[eng_idx].text)
            continue
        for key in ["none", "ho", "hang"]:
            cur_eng = list(map(lambda x: x.text, contents[eng_idx].find_elements(By.CLASS_NAME, key)))
            cur_kor = list(map(lambda x: x.text, contents[kor_idx].find_elements(By.CLASS_NAME, key)))
            if len(cur_eng)==len(cur_kor):
                eng_line+=cur_eng
                kor_line+=cur_kor
            else:
                return law
    return pd.DataFrame(zip([cate]*len(eng_line), [law]*len(kor_line), eng_line, kor_line), columns=["category", "lawName", "ENG", "KOR"])
    
result = parsing("분류", "법령명")
result

,category,lawName,ENG,KOR
0,분류,법령명,The purpose of this Act is to enhance inter-Ko...,이 법은 개성공업지구의 개발ㆍ운영의 지원 및 개성공업지구에 투자하거나 출입ㆍ체류하는...
1,분류,법령명,The terms used in this Act are defined as foll...,"이 법에서 사용하는 용어의 정의는 다음과 같다. <개정 2009. 1. 30., 2..."
2,분류,법령명,1. The term “Gaeseong Industrial Complex” or “...,1. “개성공업지구”란 남한과 북한 사이의 합의에 따라서 북한의 개성지역 일대에서 ...
3,분류,법령명,2. The term “GIC developer” means a South Kore...,2. “개성공업지구 개발업자”란 「남북교류협력에 관한 법률」에 따른 협력사업의 승인...
4,분류,법령명,3. The term “GIC management organ” means a cor...,3. “개성공업지구 관리기관”이란 개성공업지구의 관리ㆍ운영을 위하여 북한의 「개성공...
...,...,...,...,...
108,분류,법령명,(1) If deemed necessary to support the develop...,"① 정부는 개성공업지구의 개발과 관리ㆍ운영을 지원하고, 남북경제협력을 증진시키기 위..."
109,분류,법령명,(2) The Government may provide necessary expen...,② 정부는 제1항을 위하여 필요한 비용을 지원할 수 있다.
110,분류,법령명,(3) The dispatch of public officials under par...,"③ 제1항의 공무원의 파견은 「남북관계 발전에 관한 법률」에서 정한 바에 따르고, ..."
111,분류,법령명,(1) (Enforcement Date) This Act shall enter in...,① (시행일) 이 법은 공포 후 3개월이 경과한 날부터 시행한다.
